In [1]:
import os

dir_notebook = os.getcwd()
print(dir_notebook)

/home/david/proyectos4/repositorios/Proyectos-SGBA1/Pruebas/Modelos/precio_peninsula


In [2]:
ruta_trabajo = '../../../'

# Cambiar el directorio de trabajo
os.chdir(ruta_trabajo)

print("Directorio de trabajo actual:", os.getcwd())

Directorio de trabajo actual: /home/david/proyectos4/repositorios/Proyectos-SGBA1


In [3]:

import pandas as pd
ruta_datos = os.path.join(os.getcwd(), 'data/processed/datos_precio/clima_precio_merged_recortado.parquet')
df = pd.read_parquet(ruta_datos)


In [4]:
df.head()

,timestamp,€/kwh,fecha,tmed_3196,tmin_3196,tmax_3196,velmedia_3196,racha_3196,sol_3196,presMax_3196,...,horaPresMax_cos_8175,horaPresMin_varias_8175,horaPresMin_minutos_8175,horaPresMin_sin_8175,horaPresMin_cos_8175,prec_inapreciable_8175,prec_valor_8175,prec_log_8175,dir_sin_8175,dir_cos_8175
0,2014-11-03 13:00:00,0.11579,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
1,2014-11-03 14:00:00,0.10764,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
2,2014-11-03 15:00:00,0.10876,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
3,2014-11-03 16:00:00,0.10854,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648
4,2014-11-03 17:00:00,0.11423,2014-11-03,11.5,8.0,15.0,6.1,17.5,1.1,938.3,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.984808,-0.173648


In [5]:
# List of particles to keep in column names
lista_particulas_mantener = [
    "timestamp",
    "€/kwh",
    "tm", 
    "viento",
    "sol",
    # "fecha",
    "anio",
    # "mes",
    # "dia",
    # "dia_semana",
    # "estacion"
]

# Create a copy of the original dataframe
df_reducido = df.copy()

# Drop columns that don't contain any of the particles
for column in df.columns:
    if not any([particula in column for particula in lista_particulas_mantener]):
        df_reducido.drop(column, axis=1, inplace=True)

# Rename columns for Prophet compatibility
df_reducido = df_reducido.rename(columns={"€/kwh": 'y', "timestamp": 'ds'})
df_reducido.head()
df = df_reducido

In [6]:
# Separar en entrenamiento y validación
ultimo_anio = df['anio'].max()
df_entrenamiento = df[df['anio'] < ultimo_anio].copy()
df_validacion = df[df['anio'] == ultimo_anio].copy()

In [7]:
from prophet import Prophet

regressor_cols = [col for col in df_entrenamiento.columns if col not in ['y', 'ds']]

model_prophet = Prophet()

for col in regressor_cols:
        model_prophet.add_regressor(col)

/home/david/proyectos4/p4_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Ensure the training data has 'ds' and 'y' columns
df_entrenamiento.head()

,ds,y,tmed_3196,tmin_3196,tmax_3196,sol_3196,anio,horatmin_varias_3196,horatmin_minutos_3196,horatmin_sin_3196,...,tmax_8175,sol_8175,horatmin_varias_8175,horatmin_minutos_8175,horatmin_sin_8175,horatmin_cos_8175,horatmax_varias_8175,horatmax_minutos_8175,horatmax_sin_8175,horatmax_cos_8175
0,2014-11-03 13:00:00,0.11579,11.5,8.0,15.0,1.1,2014.0,0.0,150.0,0.608761,...,19.6,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308
1,2014-11-03 14:00:00,0.10764,11.5,8.0,15.0,1.1,2014.0,0.0,150.0,0.608761,...,19.6,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308
2,2014-11-03 15:00:00,0.10876,11.5,8.0,15.0,1.1,2014.0,0.0,150.0,0.608761,...,19.6,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308
3,2014-11-03 16:00:00,0.10854,11.5,8.0,15.0,1.1,2014.0,0.0,150.0,0.608761,...,19.6,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308
4,2014-11-03 17:00:00,0.11423,11.5,8.0,15.0,1.1,2014.0,0.0,150.0,0.608761,...,19.6,9.1,0.0,420.0,0.965926,-0.258819,0.0,820.0,-0.422618,-0.906308


In [9]:
columns_to_check = ['y', 'ds', 'timestamp', '€/kwh']
present_columns = [col for col in columns_to_check if col in df.columns]
print("Columns present in df_entrenamiento:", present_columns)

Columns present in df_entrenamiento: ['y', 'ds']


In [10]:
import dagshub
dagshub.init(repo_owner='auditoria.SGBA1', repo_name='Proyectos-SGBA1', mlflow=True)

Accessing as auditoria.SGBA1

Initialized MLflow to track repo "auditoria.SGBA1/Proyectos-SGBA1"

Repository auditoria.SGBA1/Proyectos-SGBA1 initialized!

In [11]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import mlflow

mlflow.set_experiment("precio/")

with mlflow.start_run(run_name="Prophet_Model_Run"):

    model_prophet = Prophet()

    for col in regressor_cols:
            model_prophet.add_regressor(col)

    model_prophet.fit(df_entrenamiento[['ds', 'y'] + regressor_cols])

    # Create future dataframe for prediction (using validation data timeframe)
    future = df_validacion[['ds'] + regressor_cols].copy()

    # Make predictions
    forecast = model_prophet.predict(future)

    # Merge predictions with actual values
    forecast_filtered = forecast[['ds', 'yhat']].merge(df_validacion[['ds', 'y']], on='ds')

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(forecast_filtered['y'], forecast_filtered['yhat']))

    # Calculate R2
    r2 = r2_score(forecast_filtered['y'], forecast_filtered['yhat'])

    print(f'RMSE: {rmse:.4f}')
    print(f'R2 Score: {r2:.4f}')

    # Log metrics
    mlflow.log_metric("val_rmse", rmse)
    mlflow.log_metric("val_r2", r2)
    
    # Log model
    mlflow.prophet.log_model(model_prophet, "prophet_model")

19:41:40 - cmdstanpy - INFO - Chain [1] start processing
19:44:49 - cmdstanpy - INFO - Chain [1] done processing
/home/david/proyectos4/p4_venv/lib/python3.10/site-packages/prophet/forecaster.py:1272: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['trend'] = self.predict_trend(df)


RMSE: 0.0724
R2 Score: -0.8909


2025/04/02 19:44:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Prophet_Model_Run at: https://dagshub.com/auditoria.SGBA1/Proyectos-SGBA1.mlflow/#/experiments/1/runs/5e5695f0599b4ab1b72e9025bdc4a86a
🧪 View experiment at: https://dagshub.com/auditoria.SGBA1/Proyectos-SGBA1.mlflow/#/experiments/1
